In [66]:
import pandas as pd
from bs4 import BeautifulSoup
import bs4
from multiprocessing import Pool, Lock, Value
from time import sleep
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
import base64
import numpy as np

In [11]:
df = pd.read_csv('data/kaggle_train_data_tab_new.csv', sep='\t')

In [12]:
df.head()

,Id,Prediction,Url,PageBase64
0,-9222401963271173253,0,http://lawleader.ru/docs/32/,PCFET0NUWVBFIGh0bWwgUFVCTElDICItLy9XM0MvL0RURC...
1,-9221399504663603656,0,http://alterhost.ruwww.alterhost.ruwww.user.kz...,PCFET0NUWVBFIGh0bWwgUFVCTElDICItLy9XM0MvL0RURC...
2,-9219733223042265364,0,http://www.ferio.ru/c_1-acura-legend.html,PCFET0NUWVBFIEhUTUwgUFVCTElDICItLy9XM0MvL0RURC...
3,-9218561774155397294,0,http://fshops.otzyv.ru/read.php?id=277,PCFET0NUWVBFIEhUTUwgUFVCTElDICItLy9XM0MvL0RURC...
4,-9216163564171210203,1,http://penhea895.narod.ru/familiya-shushmarche...,PCFET0NUWVBFIGh0bWwgUFVCTElDICItLy9XM0MvL0RURC...


In [86]:
def extract_features(html_page):
    html = base64.b64decode(html_page)
    soup = BeautifulSoup(html)
    if soup.title:
        title_words_cnt = len(soup.title.text.split())
    else:
        title_words_cnt = 0
    if soup.text:
        text_list = soup.text.split()
        words_cnt = len(text_list)
        mean_word_length = np.mean(list(map(len, text_list)))
    else:
        words_cnt = 0
        mean_word_length = 0
    link_cnt = len(soup.find_all('link'))
    
    return [title_words_cnt, words_cnt, mean_word_length, link_cnt]

In [87]:
html = base64.b64decode(df['PageBase64'].values[0])
soup = BeautifulSoup(html)

In [88]:
np.mean(list(map(len, soup.text.split())))

7.2619354838709675

In [89]:
mutex = Lock()
n_processed = Value('i', 0)

def func_wrapper(page):
    res = extract_features(page) 
    with mutex:
        # в этом блоке можно безопасно менять общие объекты для процессов
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return res

with Pool(processes=10) as pool:
    res = pool.map(func_wrapper, df['PageBase64'].values)


3090 objects are processed...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


7040 objects are processed...

In [90]:
X_train = np.array(res)
y_train = df['Prediction'].values

In [91]:
clf = XGBClassifier()
scores = cross_validate(clf, X_train, y_train, cv=3, scoring='f1')

In [92]:
scores['test_score'].mean()

0.866362224851648